In [1]:
import pandas as pd
import json
import numpy as np
from globals import BASE_DIR
import os
import matplotlib.pyplot as plt
import random
from evaluation_metrics import *


top_k_resample = 150
top_k_eval = 10
gridsearch = True
save_upd = True
use_subsample = True

general_models = ["BPR"]
context_models = ["LORE", "USG"]
context_models = []

available_datasets = ["brightkite", "foursquaretky", "gowalla", "snowcard", "yelp"]
available_datasets = ["snowcard", "yelp"]


#valid_popularity = "business_popularity:float"
valid_popularity = "item_pop"

recommendation_dirpart = "recommendations"

pd.options.mode.copy_on_write = True

In [2]:
# Define the datasets you want to process
def dataset_metadata(dataset, recommendation_dirpart=recommendation_dirpart):
    data = []

    recs = os.listdir(f"{BASE_DIR}{dataset}_dataset/{recommendation_dirpart}")
    if '.DS_Store' in recs:
        recs.remove('.DS_Store')

    for dir in recs:
        json_file = f"{BASE_DIR}{dataset}_dataset/{recommendation_dirpart}/{dir}/general_evaluation.json"
        
        with open(json_file, 'r') as f:
            eval_data = json.load(f)
        
        # Extract the test_result data and flatten it
        test_results = eval_data.get("test_result", {})
        test_results["directory"] = dir

        test_results["dataset"] = test_results["directory"].split("-")[0]
        test_results["model"] = test_results["directory"].split("-")
        if test_results["directory"].split("-")[1] == "debias":
            test_results["model_type"] = "debias"
            test_results["date"] = "-".join(test_results["directory"].split("-")[3:])
        
        elif test_results["directory"].split("-")[1] == "contextpoi":
            test_results["model_type"] = "contextpoi"
            test_results["date"] = "-".join(test_results["directory"].split("-")[3:])
        else:
            test_results["model_type"] = "general"
            test_results["date"] = "-".join(test_results["directory"].split("-")[2:])

        if test_results["model_type"] == "debias":
            test_results["model"] = test_results["model"][2]
        
        elif test_results["model_type"] == "contextpoi":
            test_results["model"] = test_results["model"][2]
            
        else:
            test_results["model"] = test_results["model"][1]

        if test_results["model"] == "MF":
            test_results["model_type"] = "general (via RecBole debias)"

        test_results["dataset"] = test_results["dataset"].split("_")[0]
        if test_results["model_type"] != "debias":
            data.append(test_results)

    return data


In [3]:
def unstack_recommendations(df):
    # Repeat each user_id for the length of their item_id:token list
    unstacked_df = df.explode(["item_id:token", "score"]).reset_index(drop=True)
    return unstacked_df

In [4]:
def create_base_recommendations(recommender_dir, top_k_resample=top_k_resample, top_k_eval=top_k_eval):

    with open(recommender_dir) as f:
        data = json.load(f)


    base_recommendations = []

    for user, items in data.items():
        for item in items:
            base_recommendations.append({
                "user_id:token": user,
                "item_id:token": item["item_id"],
                "score": item["score"]
            })

    base_df = pd.DataFrame(base_recommendations)
    base_df = unstack_recommendations(base_df)

    # normalize the scores
    base_df["score"] = base_df.groupby("user_id:token")["score"].transform(lambda x: (x - x.min()) / (x.max() - x.min()))


    # top k recommendations for resampling
    base_top_k_df = (
        base_df.groupby("user_id:token")
        .head(top_k_resample)
        .reset_index(drop=True)
    )

    # top k recommendations for evaluation
    base_eval_df = (
        base_df.groupby("user_id:token")
        .head(top_k_eval)
        .reset_index(drop=True)
    )

    return base_top_k_df, base_eval_df


In [5]:
def save_top_k(sorted_top_k_df, base_dir, reranking_method):
    # Create the directory if it doesn't exist
    grouped_data = (
        sorted_top_k_df.groupby("user_id:token")["item_id:token"]
        .apply(list)
        .to_dict()
    )


    save_path = os.path.join(base_dir, reranking_method)
    os.makedirs(save_path, exist_ok=True)
    file_path = os.path.join(save_path, "top_k_recommendations.json")

    with open(file_path, "w") as f:
        json.dump(grouped_data, f, indent=4)

    print(f"Saved recommendations to {file_path}")


In [6]:
def save_cp_metadata(base_dir, file, filename):
    # Create the directory if it doesn't exist
    
    save_path = os.path.join(base_dir, "cp")
    os.makedirs(save_path, exist_ok=True)
    file_path = os.path.join(save_path, f"{filename}.json")

    with open(file_path, "w") as f:
        json.dump(file, f, indent=4)

    print(f"Saved recommendations to {file_path}")

In [7]:
def rerank_upd(dataset, base_df, top_k_eval, valid_popularity, dir_to_save, calibrate_on="mean"):
    # Load user-item interaction data
    checkin_df = pd.read_csv(f"{BASE_DIR}{dataset}_dataset/processed_data_recbole/{dataset}_sample.inter", sep="\t")
    
    # Calculate item popularity
    value_counts = checkin_df["item_id:token"].value_counts().reset_index()
    value_counts.columns = ["item_id:token", "count"]
    value_counts[valid_popularity] = value_counts["count"] / len(value_counts)
    checkin_df = checkin_df.merge(value_counts[["item_id:token", valid_popularity]], on="item_id:token", how="left")

    # Assign item popularity groups
    checkin_df.sort_values(by=valid_popularity, ascending=False, inplace=True)
    item_popularity = checkin_df.drop_duplicates(subset="item_id:token", keep="first")[["item_id:token", valid_popularity]]
    
    h_group = item_popularity.head(int(len(item_popularity) * 0.2))
    h_group["item_pop_group"] = "h"
    t_group = item_popularity.tail(int(len(item_popularity) * 0.2))
    t_group["item_pop_group"] = "t"
    m_group = item_popularity[
        ~item_popularity["item_id:token"].isin(h_group["item_id:token"]) &
        ~item_popularity["item_id:token"].isin(t_group["item_id:token"])
    ]
    m_group["item_pop_group"] = "m"

    item_popularity = pd.concat([h_group, m_group, t_group])
    item_popularity.sort_values(by=valid_popularity, inplace=True, ascending=False)

    checkin_df = checkin_df.merge(item_popularity[["item_id:token", "item_pop_group"]], on="item_id:token", how="left")

    # Calculate user popularity tolerance score (UPTS)
    if calibrate_on == "mean":
        upts = checkin_df.groupby("user_id:token")[valid_popularity].mean().reset_index()
    else:
        upts = checkin_df.groupby("user_id:token")[valid_popularity].median().reset_index()

    upts.columns = ["user_id:token", "upts"]

    # Merge and calculate UPD
    merged_df = base_df.merge(upts, on="user_id:token").merge(item_popularity, on="item_id:token")
    merged_df["upd"] = (merged_df[valid_popularity] - merged_df["upts"]).abs()

    # Extract calibrated top-k recommendations
    sorted_top_k_df = (
        merged_df.sort_values(by=["user_id:token", "upd"], ascending=True)
        .groupby("user_id:token")
        .head(top_k_eval)
        .reset_index(drop=True)
    )

    return sorted_top_k_df, item_popularity, upts


## Implement CP from here: 

https://github.com/rUngruh/mitigatingPopularityBiasInMRS/blob/main/studytool/Tool-Module/scripts/LFMRecommendations/Models/mitigation.py


In [8]:
def open_ground_truth_user_group(dataset):
    # Stays the same across all models 
    train_data = pd.read_csv(f"{BASE_DIR}{dataset}_dataset/processed_data_recbole/{dataset}_sample.train.inter", sep="\t")
    test_data = pd.read_csv(f"{BASE_DIR}{dataset}_dataset/processed_data_recbole/{dataset}_sample.test.inter", sep="\t")
    valid_data = pd.read_csv(f"{BASE_DIR}{dataset}_dataset/processed_data_recbole/{dataset}_sample.valid.inter", sep="\t")
    #valid_data = pd.read_csv(f"{BASE_DIR}{dataset}_dataset/processed_data_recbole/{dataset}_sample.valid.inter", sep="\t") # originale struktur !!!
    train_data = pd.concat([train_data, valid_data])
    user_group_dir = f"{BASE_DIR}{dataset}_dataset/{dataset}_user_id_popularity.json"
    with open(user_group_dir) as f:
        user_groups = json.load(f)

    return train_data, test_data, user_groups

In [9]:
def recommender_dir_combiner(dataset, modelpart):
    top_k_dir = f"{BASE_DIR}{dataset}_dataset/{recommendation_dirpart}/{modelpart}/top_k_recommendations.json"
    recommendation_folder = f"{BASE_DIR}{dataset}_dataset/{recommendation_dirpart}/{modelpart}"
    return top_k_dir, recommendation_folder


In [10]:
def calculate_user_popularity_distributions(df, item_popularity):

    #df = df.merge(item_popularity, on="item_id:token", how="left")
    # Calculate mean, median, and variance of `business_popularity` for each user
    user_stats = df.groupby('user_id:token')[valid_popularity].agg(['mean', 'median', 'var']).reset_index()

    # Calculate normalized ratios of `item_pop_group` for each user
    pop_group_counts = df.groupby(['user_id:token', 'item_pop_group']).size().unstack(fill_value=0)

    # Normalize by row sum to get ratios
    pop_group_ratios = pop_group_counts.div(pop_group_counts.sum(axis=1), axis=0).reset_index()

    # Merge the statistics and ratios into a single DataFrame
    user_pop_ratio_df = pd.merge(user_stats, pop_group_ratios, on='user_id:token', how='left')

    # Rename columns for clarity
    user_pop_ratio_df.rename(columns={'h': 'h_ratio', 'm': 'm_ratio', 't': 't_ratio', 'mean' : 'mean_pop', 'median' : 'median_pop', 'var' : 'variance_pop'}, inplace=True)

    num_interactions = df.groupby('user_id:token').size().reset_index(name='num_interactions')

    # Merge the num_interactions into the user_pop_ratio_df
    user_pop_ratio_df = pd.merge(user_pop_ratio_df, num_interactions, on='user_id:token', how='left')

    return user_pop_ratio_df

In [11]:
def get_profile_and_recommended_ratios_for_js(test_item_groups, test_user_profile):
    recommended_ratios_nested = test_item_groups.to_dict()
    recommended_ratios = {key: list(value.values())[0] for key, value in recommended_ratios_nested.items()}

    # Extract user profile ratios
    profile_ratios = {
        'h_ratio': test_user_profile['h_ratio'].iloc[0],
        'm_ratio': test_user_profile['m_ratio'].iloc[0],
        't_ratio': test_user_profile['t_ratio'].iloc[0],
    }

    return recommended_ratios, profile_ratios


def rerank_for_user(initial_list, scores, item_popularity, user_profile, delta, k, user_id):
    """
    Perform the re-ranking for a single user using the CP algorithm.
    """
    reranked_list = []
    category_counts = {'h': 0, 'm': 0, 't': 0}  # Initialize category counts
    score_count = 0  # Tracks cumulative score

    # Iteratively build the re-ranked list for the user
    for i in range(k):
        # Calculate marginal relevance for all items for the current user
        criterion = marginal_relevances(
            score_count, scores, item_popularity, category_counts, len(reranked_list), user_profile, delta)
        
        # Exclude zero values from the criterion
        non_zero_indices = [idx for idx, value in enumerate(criterion) if value != 0]
        
        if non_zero_indices:
            # Extract the non-zero values
            non_zero_values = [criterion[idx] for idx in non_zero_indices]
            
            if all(value < 0 for value in non_zero_values):  # All remaining values are negative
                # Choose the value closest to zero
                closest_to_zero_value = min(non_zero_values, key=lambda x: abs(x))
                selected_idx = non_zero_indices[non_zero_values.index(closest_to_zero_value)]
            else:
                # Choose the maximum value (default behavior)
                selected_idx = non_zero_indices[np.argmax(non_zero_values)]
        else:
            # Fallback: All values are zero
            selected_idx = np.argmax(criterion)  # Default to the first max zero


        # Update score and add selected item to re-ranked list
        score_count += scores[selected_idx]
        reranked_list.append(initial_list[selected_idx])
       
        # Update category counts
        category_counts[item_popularity[selected_idx]] += 1

        # Remove selected item from the lists
        del initial_list[selected_idx]
        del scores[selected_idx]
        del item_popularity[selected_idx]

    return reranked_list


def marginal_relevances(score_count, item_scores, item_popularities, category_counts, list_len, user_profile, delta):
    """
    Computes the marginal relevance, the criterion for CP
    """
    relevances = np.zeros(len(item_scores))
    recommendation_counts = pd.DataFrame({'h_ratio': [category_counts['h']],
                                           'm_ratio': [category_counts['m']],
                                           't_ratio': [category_counts['t']]})
    computed_categories = set()

    for i, (score, popularity) in enumerate(zip(item_scores, item_popularities)):
        
        if popularity in computed_categories:
            continue  # Avoid duplicate calculations for the same popularity class

        #print(f"Item {i}: score={score}, popularity={popularity}")
        computed_categories.add(popularity)

        # Increment the count temporarily
        recommendation_counts[popularity + '_ratio'] += 1
        recommendation_ratios = recommendation_counts / (list_len + 1)  # Normalize counts

        #print(f"Iteration {list_len + 1}: Temporary recommendation ratios: {recommendation_ratios}")

        # Compute marginal relevance with adjusted formula
        rec_ratios, profile_ratios = get_profile_and_recommended_ratios_for_js(recommendation_ratios, user_profile)
        js_divergence = jensen_shannon(profile_ratios, rec_ratios)
        
        relevances[i] = (1 - delta) * (score_count + score) - delta * js_divergence

        recommendation_counts[popularity + '_ratio'] -= 1
    return relevances



def get_individual_user_data(df, user_profiles, user_id):
    test_user_id = user_id
    test_recs = df.loc[df["user_id:token"] == test_user_id]["item_id:token"].values.tolist()   
    test_scores = df.loc[df["user_id:token"] == test_user_id]["score"].values.tolist()
    test_item_pops = df.loc[df["user_id:token"] == test_user_id][valid_popularity].values.tolist()
    test_item_groups = df.loc[df["user_id:token"] == test_user_id]["item_pop_group"].values.tolist()
    test_user_profile = user_profiles.loc[user_profiles["user_id:token"] == test_user_id]
    return test_scores, test_recs, test_item_pops, test_item_groups, test_user_id, test_user_profile


def rerank_cp_all_users(df, user_profiles, top_k_eval, delta):
    reranked_results = {}
    
    for i, user_id in enumerate(df['user_id:token'].unique()):
        test_scores, test_recs, test_item_pops, test_item_groups, test_user_id, test_user_profile = get_individual_user_data(df, user_profiles, user_id)
        reranked_list = rerank_for_user(test_recs, test_scores, test_item_groups, test_user_profile, delta, top_k_eval, test_user_id)
        reranked_results[test_user_id] = reranked_list
    
        

    # Create the DataFrame **after** the loop
    cp_results = pd.DataFrame([reranked_results]).T.reset_index()
    cp_results.columns = ["user_id:token", "item_id:token"]
    cp_reranked_df = cp_results.explode('item_id:token').reset_index(drop=True)
    
    return cp_reranked_df





In [12]:
def sample_user_groups(user_groups, sample_size=100):
    sampled_groups = {}
    for group, ids in user_groups.items():
        if len(ids) >= sample_size:
            sampled_groups[group] = random.sample(ids, sample_size)
        else:
            print(f"Warning: Group '{group}' has less than {sample_size} users. Sampling all users.")
            sampled_groups[group] = ids
    return sampled_groups

In [13]:
def calculate_group_ratios(user_groups, df):
    """Use for plotting"""
    group_results = {}

    for group_name, user_ids in user_groups.items():

        # Filter reranked_df_user and user_profiles for the current user group
        reranked_group_df = df.loc[df['user_id:token'].isin(user_ids)]

        # Calculate means for each ratio

        # if 't_ratio' not in reranked_group_df.columns:
        #     reranked_group_df['t_ratio'] = 0  # or some other default value

        means = reranked_group_df[['h_ratio', 'm_ratio', 't_ratio']].mean().to_dict()

        # Save results
        group_results[group_name] = means

    return group_results


In [14]:
def get_extreme(results, metric, mode='max', use_abs=False):

    """
    Finds the delta that gives the extreme value (argmax or argmin) for a given metric across all groups.
    
    Parameters:
    - results (dict): The nested results dictionary.
    - metric (str): The metric to compute the extreme for (e.g., 'ndcg', 'arp', 'poplift', 'js_divergence').
    - mode (str): Either 'max' (default) or 'min' to compute the argmax or argmin.
    - use_abs (bool): Whether to compute extremes based on the absolute value of the metric.
    
    Returns:
    - dict: A dictionary with group names as keys and the corresponding delta and extreme value.
    """
    if mode not in ['max', 'min']:
        raise ValueError("Mode must be either 'max' or 'min'.")
    
    extreme_dict = {}
    compare = max if mode == 'max' else min
    extreme_value_init = float('-inf') if mode == 'max' else float('inf')
    
    for group_name in next(iter(results.values())).keys():  # Get group names from the first delta
        extreme_value = extreme_value_init
        best_delta = None
        
        for delta, group_data in results.items():
            if metric in group_data[group_name]:  # Check if the metric exists
                value = abs(group_data[group_name][metric]) if use_abs else group_data[group_name][metric]
                if compare(value, extreme_value) == value:
                    extreme_value = value
                    best_delta = delta
        
        extreme_dict[group_name] = best_delta
    
    return extreme_dict


In [15]:
def cp_gridsearch(base_resample, user_profiles, top_k_eval, item_popularity, train_data, test_data, user_groups, upts, recommendation_folder):
    deltas = [0, 0.1, 0.3, 0.5, 0.7, 0.9, 1]
    results = {}
    gridsearch_best_deltas = {}
    for i, delta in enumerate(deltas):
        results[delta] = {}
        reranked_df = rerank_cp_all_users(base_resample, user_profiles, top_k_eval, delta=delta)
        reranked_df = reranked_df.merge(item_popularity, on="item_id:token")
        base_arp_scores, base_poplift_scores, calibrated_arp_scores, calibrated_poplift_scores, base_ndcg_scores, calibrated_ndcg_scores = calculate_deltas(test_data, base_resample, reranked_df, item_popularity, upts, valid_popularity, top_k_eval)
        reranked_df_user = calculate_user_popularity_distributions(reranked_df, item_popularity)
        user_profiles = calculate_user_popularity_distributions(train_data, item_popularity)

        group_hmt_means_up = calculate_group_ratios(user_groups, user_profiles)
        group_hmt_means_reranked = calculate_group_ratios(user_groups, reranked_df_user)


        group_means = evaluation_user_group_means(calibrated_ndcg_scores, calibrated_arp_scores, calibrated_poplift_scores, user_groups)
        for group_name, user_ids in user_groups.items():
            # Filter reranked_df_user and user_profiles for the current user group
            #reranked_group_df = reranked_df_user.loc[reranked_df_user['user_id:token'].isin(user_ids)]
            #user_profile_group_df = user_profiles.loc[user_profiles['user_id:token'].isin(user_ids)]
            js = jensen_shannon(group_hmt_means_up[group_name], group_hmt_means_reranked[group_name])
            harmonic_mean = group_means[group_name]['ndcg_mean'] * (1 - js) / (group_means[group_name]['ndcg_mean'] + (1 - js))

            results[delta][group_name] = {
                "js_divergence": js,
                "harmonic_mean": harmonic_mean,
                "ndcg":  group_means[group_name]['ndcg_mean'],
                "arp": group_means[group_name]['arp_mean'],
                "poplift": group_means[group_name]['poplift_mean']
            }

    gridsearch_best_deltas["ndcg"] = get_extreme(results, 'ndcg')
    gridsearch_best_deltas["arp"] = get_extreme(results, 'arp', mode='min')
    gridsearch_best_deltas["harmonic_mean"] = get_extreme(results, 'harmonic_mean')
    gridsearch_best_deltas["js"] = get_extreme(results, 'js_divergence', mode='min')
    gridsearch_best_deltas["poplift"] = get_extreme(results, 'poplift', mode='min', use_abs=True)

    save_cp_metadata(recommendation_folder, gridsearch_best_deltas, "gridsearch_best_deltas")
    save_cp_metadata(recommendation_folder, results, "gridsearch_results")

    return gridsearch_best_deltas
            




In [16]:
for dataset in available_datasets:
    data = dataset_metadata(dataset)
    for result in data:
        try:
            print(f"Processing model {result['model']} on dataset {result['dataset']}")
            baseline_topk_dir, basedir = recommender_dir_combiner(dataset, result["directory"])
            train_data, test_data, user_groups = open_ground_truth_user_group(dataset)
            base_resample, base_eval = create_base_recommendations(baseline_topk_dir, top_k_resample=top_k_resample, top_k_eval=top_k_eval)

            upd_eval, item_popularity, upts = rerank_upd(
                dataset, base_resample, top_k_eval, valid_popularity, dir_to_save=basedir, calibrate_on="mean")

            if save_upd:
                save_top_k(upd_eval, basedir, "upd")

            ##### Using a smaller subsample for testing deltas in cp
            user_groups = sample_user_groups(user_groups, sample_size=50) # for testing
            sampled_user_ids = set(id_ for group in user_groups.values() for id_ in group) # for testing
            dataframes_to_filter = [train_data, test_data, base_resample, base_eval, upts] # for testing
            filtered_dataframes = [df.loc[df['user_id:token'].isin(sampled_user_ids)] for df in dataframes_to_filter] # for testing
            train_data, test_data, base_resample, base_eval, upts = filtered_dataframes # for testing
            #### End of using a smaller subsample for testing deltas in cp

            all_user_ids = set(user_groups['high']) | set(user_groups['medium']) | set(user_groups['low'])
            user_groups['all'] = list(all_user_ids)



            base_resample = base_resample.merge(item_popularity, on="item_id:token")
            test_data = test_data.merge(item_popularity, on="item_id:token", how="left")
            train_data = train_data.merge(item_popularity, on="item_id:token", how="left")
            user_profiles = calculate_user_popularity_distributions(train_data, item_popularity)


            if gridsearch:
                cp_gridsearch_best_deltas = cp_gridsearch(base_resample, user_profiles, top_k_eval,item_popularity, train_data, test_data, user_groups, upts, basedir)

            else:
                cp_gridsearch_best_deltas = json.load(open(f"{basedir}/cp/gridsearch_best_deltas.json"))
        except Exception as e:
            print(f"Error processing model {result['model']} on dataset {result['dataset']}: {e}")
            continue




Processing model BPR on dataset brightkite


/var/folders/9q/p0b6nwnd5071040x3849mdvc0000gn/T/ipykernel_96319/1139550573.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  h_group["item_pop_group"] = "h"
/var/folders/9q/p0b6nwnd5071040x3849mdvc0000gn/T/ipykernel_96319/1139550573.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t_group["item_pop_group"] = "t"
/var/folders/9q/p0b6nwnd5071040x3849mdvc0000gn/T/ipykernel_96319/1139550573.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

Saved recommendations to /Volumes/Forster Neu/Masterarbeit Data/brightkite_dataset/recommendations/brightkite_sample-BPR-Oct-26-2024_15-21-48/upd/top_k_recommendations.json
Group: high - nDCG: 0.03463; ARP: 0.02313; Poplift: -0.54128
Group: medium - nDCG: 0.01463; ARP: 0.02356; Poplift: -0.43429
Group: low - nDCG: 0.04036; ARP: 0.02320; Poplift: -0.27420
Group: all - nDCG: 0.02987; ARP: 0.02330; Poplift: -0.41659
**************************************************
Group: high - nDCG: 0.03463; ARP: 0.02331; Poplift: -0.53749
Group: medium - nDCG: 0.01492; ARP: 0.02358; Poplift: -0.43358
Group: low - nDCG: 0.04036; ARP: 0.02320; Poplift: -0.27446
Group: all - nDCG: 0.02997; ARP: 0.02336; Poplift: -0.41517
**************************************************
Group: high - nDCG: 0.03463; ARP: 0.02343; Poplift: -0.53465
Group: medium - nDCG: 0.02070; ARP: 0.02386; Poplift: -0.42730
Group: low - nDCG: 0.03974; ARP: 0.02341; Poplift: -0.26821
Group: all - nDCG: 0.03169; ARP: 0.02356; Poplift: -0

/var/folders/9q/p0b6nwnd5071040x3849mdvc0000gn/T/ipykernel_96319/1139550573.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  h_group["item_pop_group"] = "h"
/var/folders/9q/p0b6nwnd5071040x3849mdvc0000gn/T/ipykernel_96319/1139550573.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t_group["item_pop_group"] = "t"
/var/folders/9q/p0b6nwnd5071040x3849mdvc0000gn/T/ipykernel_96319/1139550573.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

Saved recommendations to /Volumes/Forster Neu/Masterarbeit Data/brightkite_dataset/recommendations/brightkite_sample-SimpleX-Oct-26-2024_18-51-25/upd/top_k_recommendations.json
Group: high - nDCG: 0.07314; ARP: 0.02398; Poplift: -0.53704
Group: medium - nDCG: 0.01291; ARP: 0.02286; Poplift: -0.45977
Group: low - nDCG: 0.03394; ARP: 0.02340; Poplift: -0.25415
Group: all - nDCG: 0.04000; ARP: 0.02341; Poplift: -0.41699
**************************************************
Group: high - nDCG: 0.07638; ARP: 0.02395; Poplift: -0.53768
Group: medium - nDCG: 0.01291; ARP: 0.02308; Poplift: -0.45465
Group: low - nDCG: 0.03394; ARP: 0.02344; Poplift: -0.25352
Group: all - nDCG: 0.04107; ARP: 0.02349; Poplift: -0.41529
**************************************************
Group: high - nDCG: 0.07725; ARP: 0.02427; Poplift: -0.53187
Group: medium - nDCG: 0.01291; ARP: 0.02306; Poplift: -0.45482
Group: low - nDCG: 0.03972; ARP: 0.02340; Poplift: -0.25364
Group: all - nDCG: 0.04329; ARP: 0.02358; Poplift

...

/var/folders/9q/p0b6nwnd5071040x3849mdvc0000gn/T/ipykernel_96319/1139550573.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  h_group["item_pop_group"] = "h"
/var/folders/9q/p0b6nwnd5071040x3849mdvc0000gn/T/ipykernel_96319/1139550573.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t_group["item_pop_group"] = "t"
/var/folders/9q/p0b6nwnd5071040x3849mdvc0000gn/T/ipykernel_96319/1139550573.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

Saved recommendations to /Volumes/Forster Neu/Masterarbeit Data/gowalla_dataset/recommendations/gowalla_sample-BPR-Oct-26-2024_15-21-54/upd/top_k_recommendations.json
Group: high - nDCG: 0.02000; ARP: 0.00252; Poplift: 2.52884
Group: medium - nDCG: 0.00667; ARP: 0.00236; Poplift: 2.79277
Group: low - nDCG: 0.00000; ARP: 0.00289; Poplift: 5.01803
Group: all - nDCG: 0.00889; ARP: 0.00259; Poplift: 3.44654
**************************************************
Group: high - nDCG: 0.02000; ARP: 0.00252; Poplift: 2.50886
Group: medium - nDCG: 0.00667; ARP: 0.00236; Poplift: 2.79536
Group: low - nDCG: 0.00000; ARP: 0.00289; Poplift: 5.01803
Group: all - nDCG: 0.00889; ARP: 0.00259; Poplift: 3.44075
**************************************************
Group: high - nDCG: 0.02000; ARP: 0.00252; Poplift: 2.51109
Group: medium - nDCG: 0.00667; ARP: 0.00234; Poplift: 2.72574
Group: low - nDCG: 0.00000; ARP: 0.00289; Poplift: 4.99721
Group: all - nDCG: 0.00889; ARP: 0.00258; Poplift: 3.41135
***********

/var/folders/9q/p0b6nwnd5071040x3849mdvc0000gn/T/ipykernel_96319/1139550573.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  h_group["item_pop_group"] = "h"
/var/folders/9q/p0b6nwnd5071040x3849mdvc0000gn/T/ipykernel_96319/1139550573.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t_group["item_pop_group"] = "t"
/var/folders/9q/p0b6nwnd5071040x3849mdvc0000gn/T/ipykernel_96319/1139550573.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

Saved recommendations to /Volumes/Forster Neu/Masterarbeit Data/gowalla_dataset/recommendations/gowalla_sample-SimpleX-Oct-26-2024_18-51-39/upd/top_k_recommendations.json
Group: high - nDCG: 0.00000; ARP: 0.00225; Poplift: 2.82729
Group: medium - nDCG: 0.00000; ARP: 0.00212; Poplift: 3.29579
Group: low - nDCG: 0.00000; ARP: 0.00191; Poplift: 3.49209
Group: all - nDCG: 0.00000; ARP: 0.00209; Poplift: 3.20506
**************************************************
Group: high - nDCG: 0.00000; ARP: 0.00225; Poplift: 2.82729
Group: medium - nDCG: 0.00000; ARP: 0.00211; Poplift: 3.28877
Group: low - nDCG: 0.00000; ARP: 0.00191; Poplift: 3.48018
Group: all - nDCG: 0.00000; ARP: 0.00209; Poplift: 3.19874
**************************************************
Group: high - nDCG: 0.00000; ARP: 0.00225; Poplift: 2.78856
Group: medium - nDCG: 0.00000; ARP: 0.00210; Poplift: 3.27761
Group: low - nDCG: 0.00000; ARP: 0.00188; Poplift: 3.32355
Group: all - nDCG: 0.00000; ARP: 0.00208; Poplift: 3.12991
*******

FileNotFoundError: [Errno 2] No such file or directory: '/Volumes/Forster Neu/Masterarbeit Data/snoward_dataset/recommendations'

In [148]:
cp_gridsearch_best_deltas

{'ndcg': {'high': 0.3, 'medium': 1, 'low': 0.9, 'all': 0.9},
 'arp': {'high': 0, 'medium': 0, 'low': 0.1, 'all': 0},
 'harmonic_mean': {'high': 0.3, 'medium': 1, 'low': 0.9, 'all': 0.9},
 'js': {'high': 1, 'medium': 1, 'low': 1, 'all': 1},
 'poplift': {'high': 1, 'medium': 1, 'low': 1, 'all': 1}}

In [53]:
base_eval = calculate_user_popularity_distributions(base_resample, item_popularity)

In [54]:
print("BASE")
group_means = evaluation_user_group_means(base_ndcg_scores, base_arp_scores, base_poplift_scores, user_groups)
print("RE_RANKED")
group_means = evaluation_user_group_means(calibrated_ndcg_scores, calibrated_arp_scores, calibrated_poplift_scores, user_groups)

BASE
Group: high - nDCG: 0.02602; ARP: 0.02327; Poplift: -0.52907
Group: medium - nDCG: 0.02811; ARP: 0.02305; Poplift: -0.44697
Group: low - nDCG: 0.05530; ARP: 0.02309; Poplift: -0.25060
Group: all - nDCG: 0.03648; ARP: 0.02314; Poplift: -0.40888
**************************************************
RE_RANKED
Group: high - nDCG: 0.03332; ARP: 0.02790; Poplift: -0.43768
Group: medium - nDCG: 0.03324; ARP: 0.02897; Poplift: -0.30888
Group: low - nDCG: 0.07844; ARP: 0.02545; Poplift: -0.18206
Group: all - nDCG: 0.04834; ARP: 0.02744; Poplift: -0.30954
**************************************************


In [55]:

# List of algorithm DataFrames and their names
algorithm_dfs = {
    'BPR(Base)': base_df_general1,
    'BPR(Re-ranked)': calibrated_df_general1,
    'SimpleX(Re-ranked)': calibrated_df_general2,
    'SimpleX(Base)': base_df_general2,
    'LORE(Base)': base_df_context1,
    'LORE(Re-ranked)': calibrated_df_context1,
    'USG(Base)': base_df_context2,
    'USG(Re-ranked)': calibrated_df_context2
}

# Dictionary to store sorted frequencies for each algorithm
item_frequencies = {}

# Calculate frequency of each item in recommendations for each algorithm
for algorithm_name, df in algorithm_dfs.items():
    # Count item frequencies in each DataFrame
    item_counts = df['item_id:token'].value_counts(normalize=True)  # Normalize to get frequency
    sorted_counts = item_counts.sort_values(ascending=True).values  # Sort in descending order
    item_frequencies[algorithm_name] = sorted_counts  # Store sorted frequencies

# Plotting
plt.figure(figsize=(12, 6))

# Plot each algorithm's sorted item frequencies
for algorithm_name, frequencies in item_frequencies.items():
    plt.plot(frequencies, label=algorithm_name)

plt.xlabel('Items (sorted by frequency)')
plt.ylabel('Recommendation Frequency')
plt.title('Exposure Frequency of Items Recommended by Different Algorithms')
plt.legend(title='Algorithms')
plt.ylim(0, 0.045)  # Adjust as needed based on frequency range
plt.xlim(0, 8000)  # Adjust as needed based on frequency range
plt.show()


NameError: name 'base_df_general1' is not defined

## Look more fine-grained at evaluation of specific user id

In [ ]:

user_id = "311_x"

print(f"Base NDCG for user {user_id}: {base_ndcg_scores[user_id]}, Calibrated score: {calibrated_ndcg_scores[user_id]}")
print(f"Base ARP for user {user_id}: {base_arp_scores[user_id]}, Calibrated ARP: {calibrated_arp_scores[user_id]}")
print(f"Base Poplift for {user_id}: {base_poplift_scores[user_id]}, Calibrated Poplift: {calibrated_poplift_scores[user_id]}")

Base NDCG for user 311_x: 0, Calibrated score: 0
Base ARP for user 311_x: 0.11826086956521735, Calibrated ARP: 0.05739130434782604
Base Poplift for 311_x: 1.033644859813085, Calibrated Poplift: -0.013084112149532716


In [ ]:
def open_context_info():
      # Stays the same across all models 
    # train_data = pd.read_csv(f"{BASE_DIR}{dataset}_dataset/{dataset}_sample.train.inter", sep="\t")
    # test_data = pd.read_csv(f"{BASE_DIR}{dataset}_dataset//{dataset}_sample.test.inter", sep="\t")
    # valid_data = pd.read_csv(f"{BASE_DIR}{dataset}_dataset/{dataset}_sample.valid.inter", sep="\t")
    item_data = pd.read_csv(f"{BASE_DIR}{dataset}_dataset/processed_data_recbole/{dataset}_sample.item", sep="\t")
    # #valid_data = pd.read_csv(f"{BASE_DIR}{dataset}_dataset/processed_data_recbole/{dataset}_sample.valid.inter", sep="\t") # originale struktur !!!
    # train_data = pd.concat([train_data, valid_data])



    return item_data #,user_groups

In [ ]:
item_data = open_context_info()

In [ ]:
advanced_users = list(train_data.loc[train_data["item_id:token"] == "128_x"]["user_id:token"])
#user_id = advanced_users[0]

In [ ]:
def specific_user_rec_sampler(df, user_id, item_data=item_data):

    user_recs = df.loc[df['user_id:token'] == user_id]

    #user_recs = base_df_general1.loc[base_df_general1['user_id:token'] == user_id]
    user_recs = user_recs.merge(item_data, on="item_id:token", how="left")
    return user_recs

In [ ]:
user_recs = specific_user_rec_sampler(base_df_context1, user_id)

In [ ]:
import folium

def make_map(df):
    map_center = [df['lat:float'].mean(), df['lon:float'].mean()]
    map = folium.Map(location=map_center, zoom_start=5)

    # Add markers for each location
    for _, row in df.iterrows():
        folium.Marker(
            location=[row['lat:float'], row['lon:float']],
            popup=row['item_id:token'],
            tooltip=row['item_id:token']
        ).add_to(map)

    return map

In [ ]:
user_ground_truth = train_data.loc[train_data['user_id:token'] == user_id].merge(item_data, on="item_id:token", how="left")

In [ ]:
item_popularity

,item_id:token,business_popularity:float,item_pop_group
30546,1020_x,1.000000,h
2491,730_x,0.926087,h
24326,212_x,0.769565,h
1357,549_x,0.721739,h
13510,46_x,0.695652,h
...,...,...,...
14628,3821_x,0.043478,t
7016,2809_x,0.043478,t
24909,4048_x,0.043478,t
25023,429_x,0.043478,t


In [ ]:
user_recs.merge(item_popularity, on="item_id:token", how="left")

,user_id:token,item_id:token,name:token_seq,lat:float,lon:float,category_name:token_seq,business_popularity:float,item_pop_group
0,311_x,2609_x,Monsoon Chocolate,32.206491,-110.965960,"Themed Cafes, Desserts, Cafes, Restaurants, Food",0.073913,m
1,311_x,2111_x,American Eat Company,32.203908,-110.965138,"Coffee & Tea, Beer, Wine & Spirits, Grocery, F...",0.160870,h
2,311_x,1722_x,Roma Imports,32.214239,-110.948824,"Cheese Shops, Delis, Specialty Food, Restauran...",0.047826,t
3,311_x,1302_x,Welcome Diner,32.220974,-110.957513,"Restaurants, Nightlife, Comfort Food, Cocktail...",0.165217,h
4,311_x,1094_x,5 Points Market & Restaurant,32.212200,-110.969090,"Restaurants, American (New), Delis, Vegetarian...",0.121739,m
5,311_x,2052_x,Mi Nidito Restaurant,32.199964,-110.965150,"Restaurants, Mexican, Desserts, Food, Latin Am...",0.060870,m
6,311_x,1510_x,Insomnia Cookies,32.221883,-110.965287,"Ice Cream & Frozen Yogurt, Desserts, Food Deli...",0.043478,t
7,311_x,1511_x,Urban Pita,32.221883,-110.965623,"Middle Eastern, Restaurants, Greek, Falafel, F...",0.043478,t
8,311_x,61_x,OBON Sushi Bar Ramen,32.221800,-110.965922,"Sushi Bars, Restaurants, Bars, Japanese, Ramen...",0.108696,m
9,311_x,847_x,Charro Steak & Del Rey,32.221077,-110.967255,"Diners, Restaurants, Steakhouses, Mexican",0.069565,m


In [ ]:
item_data.loc[item_data["item_id:token"] == "164_x"]

,item_id:token,name:token_seq,lat:float,lon:float,category_name:token_seq
1892,164_x,Ronnie's Marcus Imax Cinema,38.527293,-90.362332,"Arts & Entertainment, Cinema"


In [ ]:
map = make_map(item_data)

In [ ]:
item_data["category_name:token_seq"].values.tolist()

['Sushi Bars, Restaurants, Japanese',
 'Korean, Restaurants',
 'Restaurants, Italian',
 'Eatertainment, Arts & Entertainment, Brewpubs, American (Traditional), Bakeries, Breweries, Food, Restaurants',
 'Sports Bars, American (New), American (Traditional), Nightlife, Bars, Restaurants',
 'Food, Beer, Wine & Spirits, Breweries',
 'Nightlife, Pubs, Event Planning & Services, Wine Bars, Bars, Gastropubs, Restaurants, Venues & Event Spaces',
 'Ice Cream & Frozen Yogurt, Coffee & Tea, Restaurants, Sandwiches, Food',
 'Restaurants, Seafood, Cajun/Creole',
 'Bars, Beer Gardens, Food, Breweries, Nightlife, Tours, Pubs, Wine Tours, Beer Tours, Hotels & Travel',
 'Venues & Event Spaces, Performing Arts, Arts & Entertainment, Hotels & Travel, Food, Convenience Stores, American (New), Beauty & Spas, Restaurants, Museums, Event Planning & Services, Hotels, Cinema, Resorts, Day Spas',
 'Steakhouses, Restaurants, Sushi Bars, Japanese',
 'Restaurants, Salad, Pakistani, Indian, Cocktail Bars, Food, Food

In [ ]:
# map

In [ ]:
# group_deltas_general1 = delta_ndcg_scores_by_group(test_data, base_df_general1, calibrated_df_general1, user_groups)
# group_deltas_general2 = delta_ndcg_scores_by_group(test_data, base_df_general2, calibrated_df_general2, user_groups)
# group_deltas_context1 = delta_ndcg_scores_by_group(test_data, base_df_context1, calibrated_df_context1, user_groups)
# group_deltas_context2 = delta_ndcg_scores_by_group(test_data, base_df_context2, calibrated_df_context2, user_groups)

In [ ]:
item_data.nunique()

item_id:token              4510
name:token_seq             4039
lat:float                  4461
lon:float                  4446
category_name:token_seq    3955
dtype: int64